In [1]:
import math
%load_ext line_profiler

This is the original function, as given in the exercise. The goal is to assess it with line_profiler and to write a faster version.

In [2]:
def primes(n=1000): 
    A = [True] * (n+1)
    
    A[0] = False
    A[1] = False
    for i in range(2, int(n**0.5)):
        if A[i]:
            for j in range(i**2, n+1, i):
                A[j] = False

    return [x for x in range(2, n) if A[x]]

In [3]:
%lprun -f primes primes(10000)